In [46]:
import pandas as pd
import numpy as np
import csv
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k
import pyarrow as pa
import pyarrow.parquet as pq
import zipfile
import csv
import scipy
import time

In [47]:
# !pip install lightfm
# !pip install fastparquet

In [48]:
train_file_path = '/Users/sumanth/final-project-team_gucci/downsampled_data/onepercent/cf_train.parquet'
train_df=pd.read_parquet(train_file_path, engine='fastparquet')
train_df=train_df.reindex(columns=['user_id', 'track_id', 'count'])
train_df

,user_id,track_id,count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,TRIDGUW128F9359DBC,6
1,85c1f87fea955d09b4bec2e36aee110927aedf9a,TRFSFAF128F42934FF,2
2,b64cdd1a0bd907e5e00b39e345194768e330d652,TRFMCVT128F92F7083,1
3,b64cdd1a0bd907e5e00b39e345194768e330d652,TRYGMSR128F93221AE,1
4,17aa9f6dbdf753831da8f38c71b66b64373de613,TRHRSBK12903CBCA78,2
...,...,...,...
497604,fff16e33847fcc82338026c3c88293c1af139a8e,TRLMZLJ128F4218745,1
497605,fff6f087923420e4e5d2b0669531b26ae351bab4,TRBTNPR12903D13765,1
497606,fff7d93d0384f02498bcbaefccd76bee486da8cc,TRBIDCO128F423B7B1,1
497607,fffa5731da86ec66f113b8eb8dc763c444110692,TRDZCBE128F931C56A,6


In [49]:
val_file_path = '/Users/sumanth/final-project-team_gucci/downsampled_data/onepercent/cf_validation.parquet'
val_df=pd.read_parquet(val_file_path,engine='fastparquet')
val_df=val_df.reindex(columns=['user_id', 'track_id', 'count'])
val_df

,user_id,track_id,count
0,0007140a3796e901f3190f12e9de6d7548d4ac4a,TRKFBTS128EF34E530,1
1,0007140a3796e901f3190f12e9de6d7548d4ac4a,TRUTMFE128F92F5D2E,1
2,002d1f1a39282b2875c08587379d9acf41866d57,TRVCNDL128F9320C3A,6
3,0102d1549242d159df98f333aee4041f96f37e98,TRIRZRK12903CE1261,1
4,0107197bd5e2521cfa1cfde3faf9a6b39e8ba9e3,TRUVRHM12903CC31F6,1
...,...,...,...
1327,ff41fd50c0ea15a4e61ff07563aa32212c92ad9a,TRRDARD12903D037FA,1
1328,ff5748549d3587444bf0a3937a3bf9e8973fe755,TROCODM128F931673E,2
1329,ff5ccf6d98ac40aafaf1aafa3a3611bb5b2e382a,TRMQIPU128F4289418,1
1330,ff852e8348cfee1d0b9d2cafe34ca000eca1cefe,TROHAXE128F422F359,2


In [50]:
test_file_path = '/Users/sumanth/final-project-team_gucci/downsampled_data/onepercent/cf_test.parquet'
test_df=pd.read_parquet(test_file_path,engine='fastparquet')
test_df=test_df.reindex(columns=['user_id', 'track_id', 'count'])
test_df

,user_id,track_id,count
0,00015189668691680bb1a2e58afde1541ec92ced,TRRGMCB128F427A490,10
1,0003d5245414b9c6244046543a06f601bfc97949,TRYUOCU128F93246AE,5
2,00071b4e8ffb68920638635660f887e8c43a3f36,TRULEEC128F4243515,1
3,000ae9c80733348ff64cd3716a27477b94578d9d,TRYYFYX128F4274F91,1
4,000f1aea4be1f190c79fcaf0ba158732b210fc0f,TRDTUJR128F428BBFA,1
...,...,...,...
13955,ffec87ea10f29a7103edd660c45c65527b1c4b9d,TRTWHOH128F4274D8A,12
13956,ffef5c05e4f25af2e8e9880af84d66936056b597,TRPUUUB12903C9AC3A,6
13957,fff040562f792c8d1caea35706d39160d0173f13,TRKCRZT12903D14001,1
13958,fff1310e43c6c37962596f528cd73d78aba17d1b,TRUWNTQ128F931258E,6


In [51]:
interactions = pd.pivot_table(test_df, index='user_id', columns='track_id', values='count')
interactions = interactions.fillna(0)
interactions

track_id,TRAACCG128F92E8A55,TRAADKW128E079503A,TRAAFYL128E07856A7,TRAAGCZ128F93210FD,TRAAGWV12903CF3709,TRAAHMD12903D034F4,TRAALAH128E078234A,TRAALYN128F1452482,TRAAMCQ128F4259A2F,TRAAQLJ128F428E870,...,TRZZJVM128E0781B78,TRZZQSK128F92EF0B9,TRZZRTB128F14603DC,TRZZSQU128F92FCDE8,TRZZSRF128F149EFBE,TRZZUBQ128F9351D51,TRZZXQI128F9326AAB,TRZZXTG128F92D5C17,TRZZZCB128F4249252,TRZZZYR128F92F0796
user_id,,,,,,,,,,,,,,,,,,,,,
00015189668691680bb1a2e58afde1541ec92ced,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0003d5245414b9c6244046543a06f601bfc97949,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00071b4e8ffb68920638635660f887e8c43a3f36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000ae9c80733348ff64cd3716a27477b94578d9d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000f1aea4be1f190c79fcaf0ba158732b210fc0f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffec87ea10f29a7103edd660c45c65527b1c4b9d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ffef5c05e4f25af2e8e9880af84d66936056b597,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fff040562f792c8d1caea35706d39160d0173f13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
(train_matrix, test_matrix) = random_train_test_split(csr_matrix(interactions.values), test_percentage=0.2)

In [53]:
rank = 50
reg_param = 0.01
top = 500

model = LightFM(loss='warp', no_components=rank, user_alpha=reg_param)

begin_fit_time = time.time()
model = model.fit(train_matrix, verbose=False)
total_fit_time = (time.time() - begin_fit_time)/60

begin_evaluation_time = time.time()
precision_at_K = precision_at_k(model, test_matrix, k=top).mean()
total_evaluation_time = (time.time() - begin_evaluation_time)/60

print('trained model with:')
print("reg_param = ", reg_param)
print("rank = ",rank)
print(f'Precision at K: {precision_at_K}')
print(f'Run time for fitting {total_fit_time} mins')
print(f'Run time for evaluation {total_evaluation_time} mins')

trained model with:
reg_param =  0.01
rank =  50
Precision at K: 0.00025191117310896516
Run time for fitting 0.0011531511942545574 mins
Run time for evaluation 0.03498030106226603 mins
